# Generative Adversarial Networks

What is a GAN?

GANs are a framework for teaching a DL model to capture the training data's distribution so we can generate new data from the same distribution. They are made of two distinct models, a *generator* and a *discriminator*. The job of the generator is to 'fake' images that look like the training images. The job of the disciminator is to look at an image and output whether or not it is a real training image or a fake image from the generator. During training, the generator is constrantly trying to 'outsmart' the disciminator by generating better and better fakes, while the discriminator is working to become a better detective and correctly classify the real and fake images. The equilibrium of this game is when the generator is generating perfect fakes that look as if they came directly from the training data, and the discriminator is left to always guess at 50% confidence that the generator output is real or fake.

Define some notations:
- $x$: data representing an image.
- $D(x)$: discriminator network which outputs the (scalar) probability that $x$ came from the training data rather than the generator. Here, since we are dealing with images the input to $D(x)$ is an image of HWC size 3x64x64. Intuitively $D(x)$ should be HIGH when $x$ comes from the training data and LOW when $x$ comes from the generator. $D(x)$ can also be thought of as a traditional binary classifier.

For the generator's notation, let $z$ be a latent space vector sampled from a standard normal distribution. $G(z)$ represents the generator function which maps the latent vector $z$ to data-space. The goal of $G$ is to estimate the distribution that the training data comes from ($p_{\mathrm{data}}$) so it can generate fake samples from that estimated distribution $p_g$.

So $D(G(z))$ is the probability (scalar) that the output of the generator $G$ is a real image. As described in Goodfellow's paper, $D$ and $G$ plays a minimax game which $D$ tries to maximize the probability it correctly classifies reals and fakes ($\log D(x)$), and $G$ tries to minimize the probability that $D$ will predict its outputs are fake ($\log(1-D(G(x)))$). From the paper, the GAN loss function is 

$$
\min_G\max_D V(D, G) = \mathbb{E}_{x \sim p_{\mathrm{data}(x)}} + \mathbb{E}_{z\sim p_z(z)}[\log(1-D(G(x)))]
$$


In theory, the solution to this minimax game is where $p_g = p_{\mathrm{data}}$, and the discriminator guesses randomly if the inputs are real or fake. However, the convergence theory of GANs is still being actively researched and in reality models do not always train to this point.

# What is a DCGAN?

A DCGAN is a direct extension of the GAN described above, except that it explicitly uses convolutional and convolutional transpose layers in the discriminator and generator, respectively. It was first described by Radford et al. in the paper 'Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks'. The discriminator is made up of strided convolution layers, batch norm layers, and LeakyReLU activations. The input is a 3x64x64 input image and the output is a scalar probability that the input is from the real data distribution. The generator is comprised of convolutional-transpose layers, batch norm layers, and ReLU activations. The input is a latent vector $z$, that is draw from a standard normal distribution and the output is a 3x64x64 RGB image. The strided conv-transpose layers allow the latent vector to be transformed into a volume with the same shape as an image. In the paper, the authors also give some tips about how to setup the optimizers, how to calculate the loss functions, and how to initialize the model weights, all of which will be explained in the coming sections.

In [1]:
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# set random seed for reproducibility
manual_seed = 999
print('Random seed: ', manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

Random seed:  999


In [2]:
# Root directory for dataset
dataroot = 'data/celeba'

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this size using
# a transformer
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode 
ngpu = 0

This is an important step because we will be using the ImageFolder dataset class, which requires there to be subdirectories in the dataset's root folder. Now we can create the dataset, create the dataloader, set the device to run on, and finally visualize some of the training data.

In [ ]:
dataset = dset.ImageFolder(root=dataroot,
                          transform=transforms.Compose([
                              transforms.Resize(image_size),
                              transforms.CenterCrop(image_size),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                          ]))

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis('off')
plt.title('Training Images')
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(), (1,2,0)))

# Implementation
With our input parameters set and the dataset prepared, we can now get into the implementation. We will start with the weight initilization strategy, then talk about the generator, discriminator, loss functions and training loop in detail.

## Weight Initialization
From the DCGAN paper, the authors specify that all model weights shall be randomly initialized from a Normal distribution with mean=0, stddev=0.02. The `weights_init` function takes an initialized model as input and reinitializes all convolutional, convolutional-transpose and batch normalization layers to meet this criteria. This function is applied to the models immediately after initialization.

In [3]:
# Custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## Generator
The generator $G$ is designed to map the latent space vector ($z$) to data-space. Since our data are images, converting $z$ to data-space means ultimately creating a RGB image with the same size as the training images (i.e. 3x64x64). In practice, this is accomplished through a series of strided two dimensional convolutional transpose layers, each paired with a 2d Batch norm layer and a relu activation. The output of the generator is fed through a `tanh` function to return it to the input data range of `[-1, 1]`. It is worth noting the existence of the batch norm functions after the conv-transpose layers, as this is a critical contribution of the DCGAN paper. These layers help with the flow of gradients during training. An image of the generator from the DCGAN paper is shown below: